In [1]:
from __future__ import division
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import fetch_mldata
import sklearn.datasets as sk_data
import math
import time
mnist = fetch_mldata('MNIST original', data_home=sk_data.get_data_home())

IMG_PIX = mnist.data.shape[1]
IMG_SIZE = int(math.sqrt(IMG_PIX))
TRAINING_SIZE = 60000
TEST_SIZE = 10000
TOTAL_SIZE = mnist.data.shape[0]
EPOCH = 50
BATCH_SIZE = 100
WINDOW_SIZE = 10000/BATCH_SIZE

print "IMG_PIX: %s\nIMG_SIZE: %s\nTOTAL_SIZE: %s\n"%(IMG_PIX, IMG_SIZE, TOTAL_SIZE)

def vectorize_y(Y):
    v_y = np.zeros((Y.shape[0], max(set(Y.flatten()))+1))
    print v_y.shape
    for i in range(len(Y)):
        v_y[i][Y[i]] = 1.0
    return v_y

def shuffle(X, Y):
    Y = np.array([Y.flatten()]).T
    A = np.concatenate((X, Y),axis=1)
    np.random.shuffle(A)
    X_prime, Y_prime = np.hsplit(A, [-1])
#     if (set(Y_prime.flatten())!=set([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0])):
#         return shuffle(X,Y)
    Y_prime = Y_prime.reshape(Y_prime.shape[0],)
    return X_prime, Y_prime

def prepare_data(_mnist):
    _mnist.data = _mnist.data.astype(np.float32)
    _mnist.data = np.multiply(_mnist.data, 1.0 / 255.0)
    _mnist.target = _mnist.target.astype(np.uint8)
    train_xx, test_xx = np.vsplit(_mnist.data, [TRAINING_SIZE])
    train_yy, test_yy = np.array_split(_mnist.target, [TRAINING_SIZE])
    return train_xx, test_xx, train_yy, test_yy

print type(mnist.target[0])

/Users/NatchaS/anaconda/envs/tensorflow/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


IMG_PIX: 784
IMG_SIZE: 28
TOTAL_SIZE: 70000

<type 'numpy.float64'>


In [2]:
import fftf
import ffbn

HIDDEN_1 = 64
HIDDEN_2 = 64
ALPHA = 0.5
USING_BATCH_NORM = False

if (USING_BATCH_NORM):
    import ffbn as ff
else: 
    import fftf as ff

train_x, test_x, train_y, test_y = prepare_data(mnist)
print "training =>", train_x.shape, train_y.shape
print "testing =>", test_x.shape, test_y.shape
print ff

with tf.Graph().as_default(), tf.device('/cpu:0'):      
    img_input = tf.placeholder(tf.float32, shape=[BATCH_SIZE, IMG_PIX])
    label_input = tf.placeholder(tf.int32, shape=[BATCH_SIZE])
    model = None
    if (USING_BATCH_NORM):
        model = ff.inference(img_input, HIDDEN_1, HIDDEN_2, batch_norm_ver=None)
    else:
        model = ff.inference(img_input, HIDDEN_1, HIDDEN_2)
    loss_f = ff.loss(model, label_input)
    # 'adagrad', 'adam', 'momentum', 'default'
    training_model = ff.training(loss_f, ALPHA, optimizer="momentum", momentum=0.1)
    eval_f = ff.evaluation(model, label_input)
    init = tf.initialize_all_variables()
    
    loss_arr = []
    eval_arr = []
    output = []
    
    done = False
    
    with tf.Session(config=tf.ConfigProto(allow_soft_placement = True)) as sess:
        sess.run(init)
        moving_idx = 0
        moving_avg = np.zeros(WINDOW_SIZE)
        still_moving = 1
        t0 = time.time()
        t1 = t0
        for i in xrange(EPOCH):
            #print "EPOCH:", i+1
            acc_sum = 0
            train_x, train_y = shuffle(train_x, train_y)
            for j in xrange(0, TRAINING_SIZE, BATCH_SIZE):
                z = j + BATCH_SIZE
                train_x_batch = train_x[j:z]
                train_y_batch = train_y[j:z]
                _, loss, correct = sess.run([training_model, loss_f, eval_f], feed_dict={
                    img_input: train_x_batch,
                    label_input: train_y_batch
                })
                
                moving_avg[moving_idx] = correct/len(train_x_batch)
                moving_idx += 1
                if (still_moving and moving_idx >= WINDOW_SIZE):
                    moving_idx = 0
                    still_moving = 0
                elif (moving_idx >= WINDOW_SIZE):
                    moving_idx = 0
                
                sum = moving_avg.sum()
                if (still_moving):
                    wind_avg = sum/(moving_idx*BATCH_SIZE) * 100
                else:
                    wind_avg= sum/WINDOW_SIZE
                #wind_avg=correct/BATCH_SIZE
                
                if(j%1000)==0 or j+BATCH_SIZE == TRAINING_SIZE:
                    print("training %d - %d at 00 clocks and %s sec : classification %s / %s = %s cross entropy error %s"%(z-WINDOW_SIZE,z,time.time()-t0,sum,WINDOW_SIZE,wind_avg,loss))

                if (wind_avg >= 0.97):
                    t1 = time.time()-t0
                    print("training %d - %d at 00 clocks and %s sec : classification %s / %s = %s cross entropy error %s"%(z-WINDOW_SIZE,z,t1,sum,WINDOW_SIZE,wind_avg,loss))
                    done = True
                    break;
                    
            if (done):
                for i in output:
                    print i
                print "test_sample_start:0"
                print "test_sample_end:%s"%(TEST_SIZE)
                print "interval between reports:%s"%(BATCH_SIZE)
                print "train_sec:%s"%(t1)
                break;
        num_correct = 0
        t0 = time.time()
        for k in xrange(0, TEST_SIZE, BATCH_SIZE):
            eval_x, eval_y = test_x[k:k+BATCH_SIZE], test_y[k:k+BATCH_SIZE]
            if (len(eval_x)!=BATCH_SIZE): 
                break
            correct = sess.run(eval_f, feed_dict={
                img_input: eval_x,
                label_input: eval_y
            })
            num_correct += correct
        t1 = time.time()-t0
        print "test_sec:%s"%(t1)
        print "test_correct:%s"%(num_correct/TEST_SIZE)
        print "samples for testing:%s"%(TEST_SIZE)
        print "hidden units:%s"%(HIDDEN_1)
        print "mini batch size:%s"%(BATCH_SIZE)
        print "average window size:%s"%(WINDOW_SIZE)
        print "samples for training:%s"%(TRAINING_SIZE)
        print "language:python"
        print "output_classes:10"
        print "pixels per image:784"
        print "library:TensorFlow"
        print "processor:i5-7500"
        #print "early stop: true"
                    
                



training => (60000, 784) (60000,)
testing => (10000, 784) (10000,)
<module 'fftf' from 'fftf.pyc'>


/Users/NatchaS/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:42: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


training 0 - 100 at 00 clocks and 0.523190021515 sec : classification 0.11 / 100.0 = 0.11 cross entropy error 2.30056
training 1000 - 1100 at 00 clocks and 0.56488609314 sec : classification 3.78 / 100.0 = 0.343636363636 cross entropy error 1.46168
training 2000 - 2100 at 00 clocks and 0.6115899086 sec : classification 8.5 / 100.0 = 0.404761904762 cross entropy error 1.30723
training 3000 - 3100 at 00 clocks and 0.651308059692 sec : classification 14.71 / 100.0 = 0.474516129032 cross entropy error 1.09581
training 4000 - 4100 at 00 clocks and 0.711420059204 sec : classification 22.16 / 100.0 = 0.540487804878 cross entropy error 0.593504
training 5000 - 5100 at 00 clocks and 0.760569095612 sec : classification 30.48 / 100.0 = 0.597647058824 cross entropy error 0.721819
training 6000 - 6100 at 00 clocks and 0.807677030563 sec : classification 38.67 / 100.0 = 0.63393442623 cross entropy error 0.36779
training 7000 - 7100 at 00 clocks and 0.847009897232 sec : classification 47.09 / 100.0 =

In [3]:
# from tensorflow.examples.tutorials.mnist import input_data


# ds = input_data.read_data_sets("data", validation_size=0)

# print ds.train.epochs_completed
# print ds.train.images.shape

# print train_y[0]==ds.train.labels[1]
# print set(ds.train.images[1]) == set(train_x[0])

# plt.subplot(3,3,1)
# plt.imshow(train_x[0].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# plt.subplot(3,3,2)
# plt.imshow(train_x[1].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# plt.subplot(3,3,3)
# plt.imshow(train_x[2].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# plt.subplot(3,3,4)
# plt.imshow(train_x[3].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# plt.subplot(3,3,5)
# plt.imshow(train_x[4].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# plt.subplot(3,3,6)
# plt.imshow(train_x[5].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# plt.subplot(3,3,7)
# plt.imshow(train_x[6].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# plt.subplot(3,3,8)
# plt.imshow(train_x[7].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# plt.subplot(3,3,9)
# plt.imshow(train_x[8].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')

# # plt.subplot(1,2,2)
# # plt.imshow(ds.train.images[3].reshape((28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
# # print train_y[2]
# # print ds.train.labels[3]

# print type(ds.train.labels[0])
# print train_y[:10]





